# 📊 3D 글로벌 캘리브레이션 분석 (Global Calibration & IV Surface)

이 노트북은 **여러 만기일(Multi-Maturity)**의 옵션 데이터를 동시에 활용하여,
각 모델이 **전체 내재 변동성 표면(IV Surface)**을 얼마나 잘 설명하는지 분석합니다.

## 📌 핵심 개념
| 축 | 변수 | 설명 |
|:---:|:---:|:---|
| **X** | Moneyness ($K/S_0$) | 행사가격의 상대적 위치 (1.0 = ATM) |
| **Y** | Time to Maturity ($T$) | 잔존 만기 (연 단위) |
| **Z** | Implied Volatility ($\sigma$) | 내재 변동성 |

---

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# =============================================================================
# [MUST BE FIRST] Fix OpenMP Duplicate Library Error
# =============================================================================
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Core Libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import torch
import sys
import gc
from scipy.stats import norm
from scipy.optimize import brentq, differential_evolution

# 3D Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add parent directory to path for imports
sys.path.append(os.path.abspath('..'))

# Reload physics engine
import importlib
import src.physics_engine
importlib.reload(src.physics_engine)
from src.physics_engine import MarketSimulator

print("✅ Libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

---

## 2. 헬퍼 함수 정의 (Black-Scholes & IV Solver)

In [ ]:
def black_scholes_call_price(S, K, T, r, sigma):
    """Black-Scholes 콜 옵션 가격 계산"""
    if sigma <= 0 or T <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def implied_vol_solver(market_price, S, K, T, r, sigma_low=0.001, sigma_high=5.0):
    """Brent's method를 사용한 내재 변동성 계산"""
    if market_price <= 0 or T <= 0 or S <= 0 or K <= 0:
        return np.nan
    
    intrinsic = max(S - K * np.exp(-r * T), 0)
    if market_price < intrinsic:
        return np.nan
    
    def objective(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - market_price
    
    try:
        f_low = objective(sigma_low)
        f_high = objective(sigma_high)
        if f_low * f_high > 0:
            return np.nan
        return brentq(objective, sigma_low, sigma_high, maxiter=100)
    except:
        return np.nan

print("✅ Helper functions defined!")

---

## 3. 다중 만기일 데이터 수집 (Multi-Maturity Data Collection)

> ⚠️ **중요**: 여러 만기일의 옵션 데이터를 수집하여 3D 표면을 구성합니다.

In [ ]:
# =============================================================================
# SPY 옵션 데이터 다운로드 (다중 만기일)
# =============================================================================
ticker = "SPY"
print(f"[{ticker}] Downloading multi-maturity option chain data...")

spy = yf.Ticker(ticker)

# 현재 주가
try:
    current_price = spy.history(period="1d")['Close'].iloc[-1]
    print(f"✅ Current Price (S0): ${current_price:.2f}")
except:
    current_price = 580.0
    print(f"⚠️ Fallback Price: ${current_price}")

# 만기일 선택 (20일 ~ 150일 사이)
expirations = spy.options
today = datetime.now()
r_val = 0.04  # Risk-free rate

selected_expirations = []
for exp_date in expirations:
    exp_dt = datetime.strptime(exp_date, "%Y-%m-%d")
    days_to_expire = (exp_dt - today).days
    if 20 <= days_to_expire <= 150:
        selected_expirations.append(exp_date)

print(f"\n✅ Selected {len(selected_expirations)} expiration dates:")
for exp in selected_expirations:
    exp_dt = datetime.strptime(exp, "%Y-%m-%d")
    days = (exp_dt - today).days
    print(f"   - {exp} ({days} days)")

In [ ]:
# =============================================================================
# 다중 만기일 데이터 수집 및 IV 재계산
# =============================================================================
print("\n" + "=" * 60)
print("🔄 Collecting Multi-Maturity Data & Recalculating IV...")
print("=" * 60)

# 결과 저장용 리스트 (Strike, T, IV, Price)
surface_data = []

for exp_date in selected_expirations:
    try:
        opt_chain = spy.option_chain(exp_date)
        calls = opt_chain.calls
        
        # 유동성 필터
        calls_clean = calls[(calls['volume'] > 5) | (calls['openInterest'] > 10)].copy()
        
        # 머니니스 필터 (80% ~ 120%)
        calls_clean = calls_clean[
            (calls_clean['strike'] > current_price * 0.8) &
            (calls_clean['strike'] < current_price * 1.2)
        ]
        
        # T 계산
        exp_dt = datetime.strptime(exp_date, "%Y-%m-%d")
        T_val = max((exp_dt - today).days / 365.0, 0.01)
        
        # IV 재계산
        for _, row in calls_clean.iterrows():
            K = row['strike']
            price = row['lastPrice']
            
            recalc_iv = implied_vol_solver(price, current_price, K, T_val, r_val)
            
            if not np.isnan(recalc_iv) and 0.05 < recalc_iv < 1.5:
                moneyness = K / current_price
                surface_data.append({
                    'strike': K,
                    'moneyness': moneyness,
                    'T': T_val,
                    'iv': recalc_iv,
                    'price': price,
                    'expiration': exp_date
                })
        
        print(f"  ✅ {exp_date}: {len(calls_clean)} options processed")
        
    except Exception as e:
        print(f"  ❌ {exp_date}: Error - {e}")

# DataFrame 변환
df_surface = pd.DataFrame(surface_data)
print(f"\n✅ Total Data Points: {len(df_surface)}")
print(f"   Moneyness Range: {df_surface['moneyness'].min():.3f} ~ {df_surface['moneyness'].max():.3f}")
print(f"   Time Range: {df_surface['T'].min():.3f} ~ {df_surface['T'].max():.3f} years")
print(f"   IV Range: {df_surface['iv'].min():.4f} ~ {df_surface['iv'].max():.4f}")

---

## 4. 3D 시장 데이터 시각화 (Market IV Surface)

우선 **시장 데이터**만으로 3D 표면을 그려봅니다.

In [ ]:
# =============================================================================
# 3D Market IV Surface (Plotly)
# =============================================================================
fig = go.Figure()

# Scatter3d for market data points
fig.add_trace(go.Scatter3d(
    x=df_surface['moneyness'],
    y=df_surface['T'],
    z=df_surface['iv'],
    mode='markers',
    marker=dict(
        size=4,
        color=df_surface['iv'],
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title='IV')
    ),
    name='Market Data'
))

fig.update_layout(
    title='📈 Market Implied Volatility Surface (SPY)',
    scene=dict(
        xaxis_title='Moneyness (K/S₀)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Implied Volatility',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    width=900, height=700
)

fig.show()

---

## 5. 글로벌 캘리브레이션 (Global Calibration)

**전체 표면(Surface)**에 대해 모델을 캘리브레이션합니다.

$$\text{Loss} = \sqrt{\frac{1}{N}\sum_{i=1}^{N} (\text{Price}_{\text{model},i} - \text{Price}_{\text{market},i})^2}$$

In [ ]:
# =============================================================================
# GPU 시뮬레이터 초기화
# =============================================================================
gc.collect()
torch.cuda.empty_cache()

N_paths = 20000
dt_val = 1/252
simulator = MarketSimulator(mu=r_val, kappa=1.0, theta=0.04, xi=0.5, rho=-0.7, device='cuda')

# 데이터 준비 (NumPy arrays)
calib_strikes = df_surface['strike'].values
calib_T = df_surface['T'].values
calib_prices = df_surface['price'].values

print(f"✅ Calibration Data Ready: {len(calib_strikes)} points")

In [ ]:
# =============================================================================
# 글로벌 손실 함수 (Global Price RMSE)
# =============================================================================
def global_calibration_loss(params, model_name, market_strikes, market_T, market_prices, S0, r, dt, num_paths, simulator):
    """
    글로벌 캘리브레이션 손실 함수.
    여러 만기일의 옵션을 동시에 고려합니다.
    """
    try:
        params_dict = {'mu': r}
        
        if model_name == 'heston':
            kappa, theta, xi, rho = params
            params_dict.update({'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho})
            params_dict.update({'jump_lambda': 0, 'jump_mean': 0, 'jump_std': 0, 'vol_jump_mean': 0})
            val_type = 'heston'
        elif model_name == 'merton':
            sigma, jump_lambda, jump_mean, jump_std = params
            params_dict.update({
                'kappa': 10.0, 'theta': sigma**2, 'xi': 0.001, 'rho': 0.0,
                'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': 0
            })
            val_type = 'bates'
        elif model_name == 'bates':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std = params
            params_dict.update({
                'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho,
                'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': 0
            })
            val_type = 'bates'
        elif model_name == 'svjj':
            kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean = params
            params_dict.update({
                'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho,
                'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std,
                'vol_jump_mean': vol_jump_mean
            })
            val_type = 'svjj'
            if vol_jump_mean < 0: return 1e9
        else:
            return 1e9

        # Safety checks
        if params_dict.get('kappa', 1) < 0 or params_dict.get('theta', 1) < 0 or params_dict.get('xi', 1) < 0:
            return 1e9
        if abs(params_dict.get('rho', 0)) > 0.99:
            return 1e9

    except:
        return 1e9

    try:
        # 만기일별로 그룹화하여 시뮬레이션
        unique_T = np.unique(market_T)
        all_model_prices = []
        all_market_prices = []
        
        for T_val in unique_T:
            mask = market_T == T_val
            strikes_T = market_strikes[mask]
            prices_T = market_prices[mask]
            
            # GPU 시뮬레이션
            S_paths, _ = simulator.simulate(
                S0=S0, v0=params_dict.get('theta', 0.04),
                T=T_val, dt=dt, num_paths=num_paths,
                model_type=val_type, override_params=params_dict
            )
            S_final = S_paths[:, -1]
            
            if torch.isnan(S_final).any() or torch.mean(S_final) < 1e-3:
                return 1e9
            
            # Martingale correction
            S_mean = torch.mean(S_final)
            S_corr = S_final * (S0 / S_mean)
            
            # Price calculation
            strikes_gpu = torch.tensor(strikes_T, device=simulator.device).float()
            payoffs = torch.maximum(S_corr.unsqueeze(1) - strikes_gpu, torch.tensor(0.0, device=simulator.device))
            model_prices = (torch.mean(payoffs, dim=0) * np.exp(-r * T_val)).cpu().numpy()
            
            all_model_prices.extend(model_prices)
            all_market_prices.extend(prices_T)
        
        # Global RMSE
        all_model_prices = np.array(all_model_prices)
        all_market_prices = np.array(all_market_prices)
        rmse = np.sqrt(np.mean((all_model_prices - all_market_prices) ** 2))
        
        return rmse
        
    except:
        return 1e9

print("✅ Global Calibration Loss Function Defined!")

In [ ]:
# =============================================================================
# 글로벌 캘리브레이션 실행
# =============================================================================
base_opts = {
    'strategy': 'best1bin',
    'maxiter': 30,
    'popsize': 12,
    'tol': 0.02,
    'mutation': (0.5, 1.0),
    'recombination': 0.7,
    'workers': 1,
    'disp': True,
    'seed': 42
}

calib_args = (calib_strikes, calib_T, calib_prices, current_price, r_val, dt_val, N_paths, simulator)

# Bounds
bounds_heston = [(0.1, 10.0), (0.001, 0.2), (0.01, 2.0), (-0.95, 0.0)]
bounds_jump = [(0.1, 5.0), (-0.3, 0.1), (0.001, 0.3)]

print("=" * 60)
print("🚀 Starting GLOBAL Calibration (Multi-Maturity)")
print("=" * 60)

In [ ]:
# [1/4] Heston Global Calibration
print("\n[1/4] 🔄 Calibrating Heston Model (Global)...")
try:
    res_heston = differential_evolution(
        global_calibration_loss, bounds_heston,
        args=('heston', *calib_args),
        **base_opts
    )
    print(f"     ✅ Heston Global RMSE: {res_heston.fun:.4f}")
except Exception as e:
    print(f"     ❌ Error: {e}")
    res_heston = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7], 'fun': 1e9})()

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# [2/4] Merton Global Calibration
print("\n[2/4] 🔄 Calibrating Merton Model (Global)...")
bounds_merton = [(0.05, 0.5), (0.1, 5.0), (-0.3, 0.1), (0.001, 0.3)]
try:
    res_merton = differential_evolution(
        global_calibration_loss, bounds_merton,
        args=('merton', *calib_args),
        **base_opts
    )
    print(f"     ✅ Merton Global RMSE: {res_merton.fun:.4f}")
except Exception as e:
    print(f"     ❌ Error: {e}")
    res_merton = type('obj', (object,), {'x': [0.2, 1.0, -0.1, 0.1], 'fun': 1e9})()

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# [3/4] Bates Global Calibration
print("\n[3/4] 🔄 Calibrating Bates Model (Global)...")
bounds_bates = bounds_heston + bounds_jump
try:
    res_bates = differential_evolution(
        global_calibration_loss, bounds_bates,
        args=('bates', *calib_args),
        **base_opts
    )
    print(f"     ✅ Bates Global RMSE: {res_bates.fun:.4f}")
except Exception as e:
    print(f"     ❌ Error: {e}")
    res_bates = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7, 0.5, -0.1, 0.1], 'fun': 1e9})()

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# [4/4] SVJJ Global Calibration
print("\n[4/4] 🔄 Calibrating SVJJ Model (Global)...")
bounds_svjj = bounds_bates + [(0.001, 0.2)]
try:
    res_svjj = differential_evolution(
        global_calibration_loss, bounds_svjj,
        args=('svjj', *calib_args),
        **base_opts
    )
    print(f"     ✅ SVJJ Global RMSE: {res_svjj.fun:.4f}")
except Exception as e:
    print(f"     ❌ Error: {e}")
    res_svjj = type('obj', (object,), {'x': [2.0, 0.04, 0.5, -0.7, 0.5, -0.1, 0.1, 0.05], 'fun': 1e9})()

gc.collect()
torch.cuda.empty_cache()

print("\n" + "=" * 60)
print("✅ Global Calibration Complete!")
print("=" * 60)

---

## 6. 모델 IV Surface 생성 및 비교

In [ ]:
# =============================================================================
# 모델별 IV Surface 생성 함수
# =============================================================================
def generate_model_surface(params, model_name, df_surface, S0, r, dt, num_paths, simulator):
    """캘리브레이션된 파라미터로 모델 IV Surface 생성"""
    params_dict = {'mu': r}
    
    if model_name == 'heston':
        kappa, theta, xi, rho = params
        params_dict.update({'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho})
        params_dict.update({'jump_lambda': 0, 'jump_mean': 0, 'jump_std': 0, 'vol_jump_mean': 0})
        val_type = 'heston'
    elif model_name == 'merton':
        sigma, jump_lambda, jump_mean, jump_std = params
        params_dict.update({
            'kappa': 10.0, 'theta': sigma**2, 'xi': 0.001, 'rho': 0.0,
            'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': 0
        })
        val_type = 'bates'
    elif model_name == 'bates':
        kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std = params
        params_dict.update({
            'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho,
            'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std, 'vol_jump_mean': 0
        })
        val_type = 'bates'
    elif model_name == 'svjj':
        kappa, theta, xi, rho, jump_lambda, jump_mean, jump_std, vol_jump_mean = params
        params_dict.update({
            'kappa': kappa, 'theta': theta, 'xi': xi, 'rho': rho,
            'jump_lambda': jump_lambda, 'jump_mean': jump_mean, 'jump_std': jump_std,
            'vol_jump_mean': vol_jump_mean
        })
        val_type = 'svjj'
    
    model_ivs = []
    unique_T = df_surface['T'].unique()
    
    for T_val in unique_T:
        mask = df_surface['T'] == T_val
        strikes_T = df_surface.loc[mask, 'strike'].values
        
        try:
            S_paths, _ = simulator.simulate(
                S0=S0, v0=params_dict.get('theta', 0.04),
                T=T_val, dt=dt, num_paths=num_paths,
                model_type=val_type, override_params=params_dict
            )
            S_final = S_paths[:, -1]
            S_mean = torch.mean(S_final)
            S_corr = S_final * (S0 / S_mean)
            
            strikes_gpu = torch.tensor(strikes_T, device=simulator.device).float()
            payoffs = torch.maximum(S_corr.unsqueeze(1) - strikes_gpu, torch.tensor(0.0, device=simulator.device))
            model_prices = (torch.mean(payoffs, dim=0) * np.exp(-r * T_val)).cpu().numpy()
            
            for i, K in enumerate(strikes_T):
                iv = implied_vol_solver(model_prices[i], S0, K, T_val, r)
                model_ivs.append(iv if not np.isnan(iv) else np.nan)
        except:
            model_ivs.extend([np.nan] * len(strikes_T))
    
    return np.array(model_ivs)

print("✅ Model Surface Generator Defined!")

In [ ]:
# =============================================================================
# 모델별 IV Surface 생성
# =============================================================================
print("Generating Model IV Surfaces...")
N_viz = 100000

iv_heston = generate_model_surface(res_heston.x, 'heston', df_surface, current_price, r_val, dt_val, N_viz, simulator)
print("  ✅ Heston surface generated")

iv_merton = generate_model_surface(res_merton.x, 'merton', df_surface, current_price, r_val, dt_val, N_viz, simulator)
print("  ✅ Merton surface generated")

iv_bates = generate_model_surface(res_bates.x, 'bates', df_surface, current_price, r_val, dt_val, N_viz, simulator)
print("  ✅ Bates surface generated")

iv_svjj = generate_model_surface(res_svjj.x, 'svjj', df_surface, current_price, r_val, dt_val, N_viz, simulator)
print("  ✅ SVJJ surface generated")

---

## 7. 최종 3D 비교 시각화 (Market vs Models)

In [ ]:
# =============================================================================
# 3D Comparison: Market vs All Models
# =============================================================================
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],
           [{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=(
        f'Heston (RMSE={res_heston.fun:.2f})',
        f'Merton (RMSE={res_merton.fun:.2f})',
        f'Bates (RMSE={res_bates.fun:.2f})',
        f'SVJJ (RMSE={res_svjj.fun:.2f})'
    )
)

# Data for each subplot
models_data = [
    ('Heston', iv_heston, 1, 1),
    ('Merton', iv_merton, 1, 2),
    ('Bates', iv_bates, 2, 1),
    ('SVJJ', iv_svjj, 2, 2)
]

for model_name, model_iv, row, col in models_data:
    # Market data (gray)
    fig.add_trace(
        go.Scatter3d(
            x=df_surface['moneyness'],
            y=df_surface['T'],
            z=df_surface['iv'],
            mode='markers',
            marker=dict(size=3, color='gray', opacity=0.5),
            name='Market',
            showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )
    
    # Model data (colored)
    fig.add_trace(
        go.Scatter3d(
            x=df_surface['moneyness'],
            y=df_surface['T'],
            z=model_iv,
            mode='markers',
            marker=dict(
                size=4,
                color=model_iv,
                colorscale='Hot',
                opacity=0.8
            ),
            name=model_name,
            showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )

fig.update_layout(
    title='📊 Global Calibration: Market vs Model IV Surfaces',
    height=900, width=1100
)

fig.show()

---

## 8. 결과 요약 (Summary)

In [17]:
# =============================================================================
# 결과 요약 테이블
# =============================================================================
print("\n" + "=" * 70)
print("📊 GLOBAL CALIBRATION RESULTS (Multi-Maturity)")
print("=" * 70)

results = [
    ('Heston', res_heston.fun, 4),
    ('Merton', res_merton.fun, 4),
    ('Bates', res_bates.fun, 7),
    ('SVJJ', res_svjj.fun, 8)
]

# Sort by RMSE
results.sort(key=lambda x: x[1])

print(f"{'Rank':<6}{'Model':<12}{'Global RMSE':<15}{'# Params'}")
print("-" * 45)
for i, (name, rmse, n_params) in enumerate(results, 1):
    medal = ['🥇', '🥈', '🥉', ''][i-1] if i <= 3 else ''
    print(f"{i:<6}{name:<12}{rmse:<15.4f}{n_params} {medal}")

print("\n✅ Analysis Complete!")


📊 GLOBAL CALIBRATION RESULTS (Multi-Maturity)
Rank  Model       Global RMSE    # Params
---------------------------------------------
1     Merton      1.1667         4 🥇
2     SVJJ        1.1695         8 🥈
3     Bates       1.1708         7 🥉
4     Heston      1.6109         4 

✅ Analysis Complete!


---

## 9. Advanced Visualization (Interactive & Deep Dive)

추가 분석을 위한 심화 시각화입니다.

1. **Interactive All-in-One 3D**: 모든 모델을 한 화면에 겹쳐서 비교 (범례 클릭으로 On/Off)
2. **2D Cross-Section**: 특정 만기일(단기/중기/장기)의 단면을 잘라서 정밀 비교
3. **Error Surface**: (모델 - 시장) 오차 자체를 3D로 시각화

In [18]:
# =============================================================================
# 1. Interactive All-in-One 3D Surface
# =============================================================================
fig = go.Figure()

# Market Data (Gray Dots)
fig.add_trace(go.Scatter3d(
    x=df_surface['moneyness'], y=df_surface['T'], z=df_surface['iv'],
    mode='markers', marker=dict(size=3, color='gray', opacity=0.4),
    name='Market Data'
))

# Models (Surfaces/Markers)
model_runs = [
    ('Heston', iv_heston, 'green'),
    ('Merton', iv_merton, 'cyan'),
    ('Bates', iv_bates, 'blue'),
    ('SVJJ', iv_svjj, 'red')
]

for name, iv_data, color in model_runs:
    fig.add_trace(go.Scatter3d(
        x=df_surface['moneyness'], y=df_surface['T'], z=iv_data,
        mode='markers', marker=dict(size=2, color=color, opacity=0.7),
        name=name,
        visible=True  # All visible by default
    ))

fig.update_layout(
    title='🌐 Interactive All-in-One Comparison (Click Legend to Toggle)',
    scene=dict(
        xaxis_title='Moneyness', yaxis_title='Time', zaxis_title='Implied Volatility',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
    ),
    width=1000, height=800,
    legend=dict(itemsizing='constant')
)
fig.show()

In [19]:
# =============================================================================
# 2. 2D Cross-Section Slicing (Short/Medium/Long Term)
# =============================================================================
unique_T = np.sort(df_surface['T'].unique())
indices = [0, len(unique_T)//2, -1]
selected_T = unique_T[indices]

fig = make_subplots(rows=1, cols=3, subplot_titles=[f'T={t:.2f} yrs' for t in selected_T])

for i, T_target in enumerate(selected_T):
    mask = df_surface['T'] == T_target
    sub_df = df_surface[mask].sort_values('moneyness')
    
    # Market
    fig.add_trace(go.Scatter(
        x=sub_df['moneyness'], y=sub_df['iv'], mode='markers',
        marker=dict(color='black', size=6), name='Market' if i==0 else None,
        showlegend=(i==0)
    ), row=1, col=i+1)
    
    # Models
    # Note: Need to extract corresponding model IVs for this slice
    # Since model arrays are aligned with df_surface, we can use the same mask
    fig.add_trace(go.Scatter(x=sub_df['moneyness'], y=iv_heston[mask], mode='lines', line=dict(color='green', dash='dot'), name='Heston' if i==0 else None, showlegend=(i==0)), row=1, col=i+1)
    fig.add_trace(go.Scatter(x=sub_df['moneyness'], y=iv_bates[mask], mode='lines', line=dict(color='blue'), name='Bates' if i==0 else None, showlegend=(i==0)), row=1, col=i+1)
    fig.add_trace(go.Scatter(x=sub_df['moneyness'], y=iv_svjj[mask], mode='lines', line=dict(color='red', dash='dash'), name='SVJJ' if i==0 else None, showlegend=(i==0)), row=1, col=i+1)

fig.update_layout(title='🔪 2D Cross-Section Slicing', height=500)
fig.show()

In [20]:
# =============================================================================
# 3. Error Surface Visualization (Model - Market)
# =============================================================================
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}], [{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=['Heston Error', 'Merton Error', 'Bates Error', 'SVJJ Error']
)

error_runs = [
    (iv_heston, 1, 1),
    (iv_merton, 1, 2),
    (iv_bates, 2, 1),
    (iv_svjj, 2, 2)
]

for iv_data, r, c in error_runs:
    error = iv_data - df_surface['iv'].values
    fig.add_trace(go.Scatter3d(
        x=df_surface['moneyness'], y=df_surface['T'], z=error,
        mode='markers', marker=dict(size=3, color=error, colorscale='RdBu', showscale=True),
        name='Error'
    ), row=r, col=c)

fig.update_layout(title='🔥 Calibration Error Surfaces (Red/Blue = High Error)', height=900, width=1000)
fig.show()